In [78]:
import numpy as np
import pandas as pd
import math
import itertools
import datetime
import csv

In [79]:
def countPath(path, arr, rozmiar):
    new_path=0
    new_path+=arr[0][path[0]]
    try:
        for step in range(rozmiar-2):
            new_path+=arr[path[step]][path[step+1]]
    except:
        print('new_path+=arr[path[step]][path[step+1]]')
        print("new_path:",new_path)
        print('path',path)
        print('path.size',path.size)
        print('step',step)
        print('rozmiar-2', rozmiar-2)
        print('arr.shape',arr.shape)
    new_path+=arr[path[rozmiar-2]][0]
    return new_path

In [80]:
def wczytanieWynikow():
    the_best_solutions = pd.read_csv('Dane\\wyniki.csv', header=None, sep=';', index_col=0, names=['costam','best_length'])
    return the_best_solutions

In [81]:
def wczytanieMacierzy(rozmiar):
    file = 'Dane/data{}.txt'.format(rozmiar)
    df = pd.read_csv(file, nrows=2, header=None)
    name = df[df.index==0].values[0][0]
    n = int(df[df.index==1].values[0][0])

    df = pd.read_csv(file, skiprows=2, sep='\s+', header=None)
    arr = df.values
    return arr

In [82]:
def inicjalizacja_poczatkowych_wartosci(arr,n):
    p = 0.9
    liczba_iteracji = n
    najkrotsza_sciezka = np.random.permutation(range(1,n))
    najlepszy_wynik = countPath(najkrotsza_sciezka, arr, n)
    delta_max = 0
    poprzedni_wynik = 0
    for i in range(liczba_iteracji):
        losowa_sciezka = np.random.permutation(range(1,n))
        wynik = countPath(losowa_sciezka, arr, n)
        if wynik < najlepszy_wynik:
            najlepszy_wynik = wynik
            najkrotsza_sciezka = losowa_sciezka
        if i != 0:
            delta = wynik - poprzedni_wynik
            if delta>delta_max:
                delta_max = delta
                
        poprzedni_wynik = wynik
        
    temperatura_poczatkowa = -delta_max / np.log(p)
        
    return najkrotsza_sciezka, najlepszy_wynik

In [83]:
def zaburzonePozycje(n):
    a,b = 0,0
    while a == b:
        a,b = np.random.randint(1, high=n-1, size=2)
    return a,b

In [84]:
def swap_sciezka(zaburzona_sciezka,a,b):
    zaburzona_sciezka[a], zaburzona_sciezka[b] = zaburzona_sciezka[b], zaburzona_sciezka[a]
    return zaburzona_sciezka

In [85]:
def insert_sciezka(zaburzona_sciezka,a,b,n):
    while True:
        if b+1==a:
                a,b = zaburzonePozycje(n)
        else:
            break
    zaburzona_sciezka = np.insert(zaburzona_sciezka, a, zaburzona_sciezka[b])
    if a<b:
        zaburzona_sciezka = np.hstack((zaburzona_sciezka[:b+1],zaburzona_sciezka[b+2:]))
    else:
        zaburzona_sciezka = np.hstack((zaburzona_sciezka[:b],zaburzona_sciezka[b+1:]))
        
    return zaburzona_sciezka

In [86]:
def invert_sciezka(zaburzona_sciezka,a,b):
    if b<a:
        a, b = b, a
    while a<b:
        zaburzona_sciezka[a], zaburzona_sciezka[b] = zaburzona_sciezka[b], zaburzona_sciezka[a]
        a += 1
        b -= 1
        
    return zaburzona_sciezka

In [87]:
def obliczenia(rozmiary, liczba_powtorzen, csvwriter, struct):
    jakosc_wynikow = []    
    the_best_solutions = wczytanieWynikow()
    
    for nr, rozmiar in enumerate(rozmiary): #5 rozmiarow
        struct['rozmiar'] = rozmiar
        liczba_iteracj = [rozmiar*5, int(np.power(rozmiar, 3/2)), rozmiar*rozmiar]
        temperatury = [20*rozmiar, int(np.power(rozmiar, 3/2)), rozmiar*rozmiar]
        ilosc_powtorzen = liczba_powtorzen[nr]
        jakosc_wyniku = 0.0
        nazwa = 'data{}'.format(rozmiar)

        print('\nRozmiar:',rozmiar, end=',')
        jakosc_iteracji = []
        for liczba_it in liczba_iteracj: #5*3=15
            print(' liczba_it:', liczba_it, end=',')
            struct['liczba_it'] = liczba_it
            jakosc_temperatury = []
            for temp in temperatury: #15*3=45
                struct['temp'] = temp
                jakosc_schematow = []
                for nr_schematu in range(3): #45*3=135
                    struct['nr_schematu'] = nr_schematu
                    jakosc_permutacji = []
                    for nr_permutacji in range(3): #135*3=405
                        struct['nr_permutacji'] = nr_permutacji
                        jakosc_wyniku = 0.0
                        for powtorzenie in range(ilosc_powtorzen): #405*10=4050
                            arr = wczytanieMacierzy(rozmiar)
                            #4050*7s=28350s=427,5min=7,875h
                            wynik = obliczenieCzasuWyniku(arr, rozmiar, nr_schematu, nr_permutacji, liczba_it, temp)
                            nowa_jakosc_wyniku = the_best_solutions.loc[[nazwa],['best_length']].values[0,0]/wynik

                            if nowa_jakosc_wyniku > jakosc_wyniku:
                                jakosc_wyniku = nowa_jakosc_wyniku

                        struct['jakosc_wyniku'] = jakosc_wyniku
                        csvwriter.writerow(struct)
                        jakosc_permutacji.append(jakosc_wyniku)
                    jakosc_schematow.append(jakosc_permutacji)
                jakosc_temperatury.append(jakosc_schematow)
            jakosc_iteracji.append(jakosc_temperatury)

        jakosc_wynikow.append(jakosc_iteracji)
    
    return jakosc_wynikow

In [88]:
def obliczenieCzasuWyniku(arr, n, schemat, permutacja, liczba_iteracji, temperatura_poczatkowa):
    start = datetime.datetime.now()
    poczatkowa_sciezka, najkrotsza_dlugosc = inicjalizacja_poczatkowych_wartosci(arr,n)
    i = 0
    max_czas = 7.0
    temperatura_koncowa = 0.0001
    
    if schemat == 0:
        lambda_liniowa = temperatura_poczatkowa/(n*10)
    if schemat == 1:
        lambda_geometryczna = 0.75
    if schemat == 2:
        lambda_logarytmiczna = 2.0
    
    sciezka = poczatkowa_sciezka
    kryterium_stopu = 0
    temperatura = temperatura_poczatkowa
    poprzednia_dlugosc = najkrotsza_dlugosc
    duration = datetime.datetime.now() - start
    
    while duration.total_seconds() < max_czas and temperatura >= temperatura_koncowa:
        for m in range(liczba_iteracji): # w każdej iteracji i, proces generowania nowego rozwiązania zaburzonego jest 
                             # powrarzany s(np. n, n^2, n^2/2, n^2/4, itp.) razy dla danej wartości temperatury T_i
            a,b = zaburzonePozycje(n)
            zaburzona_sciezka = sciezka.copy()
            if permutacja == 0:
                zaburzona_sciezka = swap_sciezka(zaburzona_sciezka,a,b)
            
            elif permutacja == 1:
                zaburzona_sciezka = insert_sciezka(zaburzona_sciezka,a,b,n)
              
            else:
                zaburzona_sciezka = invert_sciezka(zaburzona_sciezka,a,b)
            
            dlugosc = countPath(zaburzona_sciezka, arr, n)
            roznica = dlugosc - poprzednia_dlugosc
            #print(roznica, '**', math.exp(-roznica/temperatura))
            if roznica <= 0:
                sciezka = zaburzona_sciezka
                poprzednia_dlugosc = dlugosc
            elif np.random.rand() < math.exp(-roznica/temperatura):
                sciezka = zaburzona_sciezka
                poprzednia_dlugosc = dlugosc
            if dlugosc < najkrotsza_dlugosc:
                najkrotsza_dlugosc = dlugosc
        
        i += 1
        
        if schemat == 1:
            temperatura = lambda_geometryczna*temperatura #schemat geometryczny
        elif schemat == 2:
            temperatura = temperatura/(1 + lambda_logarytmiczna*temperatura) #schemat logarytmiczny
        else:
            temperatura = temperatura - lambda_liniowa #schemat liniowy
            
        duration = datetime.datetime.now() - start
   
    #print('i:',i)
    #print('temperatura:', temperatura)
    #print('duration:',duration.total_seconds(), '\n')
    
    return najkrotsza_dlugosc

In [89]:
struct = {
            'rozmiar' : 0,
            'liczba_it' : 0,
            'temp' : 0,
            'nr_schematu' : 0,
            'nr_permutacji' : 0,
            'jakosc_wyniku' : 0
        }
with open('simulated_annealing.csv','w') as csvfile:
    fieldnames = ['rozmiar', 'liczba_it', 'temp', 'nr_schematu', 'nr_permutacji', 'jakosc_wyniku']
    csvwriter = csv.DictWriter(csvfile, fieldnames=fieldnames)
    csvwriter.writeheader()

    rozmiary = [21, 48, 71, 120, 323]
    liczba_powtorzen = [10, 10, 10, 10, 10]
    jakosc_wynikow = obliczenia(rozmiary, liczba_powtorzen, csvwriter, struct)


Rozmiar: 21, liczba_it: 105, liczba_it: 96, liczba_it: 441,
Rozmiar: 48, liczba_it: 240, liczba_it: 332, liczba_it: 2304,
Rozmiar: 71, liczba_it: 355, liczba_it: 598, liczba_it: 5041,
Rozmiar: 120, liczba_it: 600, liczba_it: 1314, liczba_it: 14400,
Rozmiar: 323, liczba_it: 1615, liczba_it: 5805, liczba_it: 104329,

In [90]:
df = pd.read_csv('simulated_annealing.csv')
df

,rozmiar,liczba_it,temp,nr_schematu,nr_permutacji,jakosc_wyniku
0,21,105,420,0,0,0.960951
1,21,105,420,0,1,0.960951
2,21,105,420,0,2,1.000000
3,21,105,420,1,0,1.000000
4,21,105,420,1,1,1.000000
...,...,...,...,...,...,...
400,323,104329,104329,1,1,0.642131
401,323,104329,104329,1,2,0.642131
402,323,104329,104329,2,0,0.642131
403,323,104329,104329,2,1,0.642131


In [ ]:
def obliczenieCzasuWyniku(arr, n, schemat, permutacja, liczba_iteracji, temperatura_poczatkowa):
    start = datetime.datetime.now()
    poczatkowa_sciezka, najkrotsza_dlugosc = inicjalizacja_poczatkowych_wartosci(arr,n)
    i = 0
    max_czas = 7.0
    temperatura_koncowa = 0.0001
    
    if schemat == 0:
        lambda_liniowa = temperatura_poczatkowa/(n*10)
    if schemat == 1:
        lambda_geometryczna = 0.75
    if schemat == 2:
        lambda_logarytmiczna = 2.0
    
    sciezka = poczatkowa_sciezka
    temperatura = temperatura_poczatkowa
    poprzednia_dlugosc = najkrotsza_dlugosc
    duration = datetime.datetime.now() - start
    
    while duration.total_seconds() < max_czas and temperatura >= temperatura_koncowa:
        for m in range(liczba_iteracji):
            a,b = zaburzonePozycje(n)
            zaburzona_sciezka = sciezka.copy()
            if permutacja == 0:
                zaburzona_sciezka = swap_sciezka(zaburzona_sciezka,a,b)
            
            elif permutacja == 1:
                zaburzona_sciezka = insert_sciezka(zaburzona_sciezka,a,b,n)
              
            else:
                zaburzona_sciezka = invert_sciezka(zaburzona_sciezka,a,b)
            
            dlugosc = countPath(zaburzona_sciezka, arr, n)
            roznica = dlugosc - poprzednia_dlugosc
            if roznica <= 0:
                sciezka = zaburzona_sciezka
                poprzednia_dlugosc = dlugosc
            elif np.random.rand() < math.exp(-roznica/temperatura):
                sciezka = zaburzona_sciezka
                poprzednia_dlugosc = dlugosc
            if dlugosc < najkrotsza_dlugosc:
                najkrotsza_dlugosc = dlugosc
        
        i += 1
        
        if schemat == 1:
            temperatura = lambda_geometryczna*temperatura #schemat geometryczny
        elif schemat == 2:
            temperatura = temperatura/(1 + lambda_logarytmiczna*temperatura) #schemat logarytmiczny
        else:
            temperatura = temperatura - lambda_liniowa #schemat liniowy
            
        duration = datetime.datetime.now() - start
    
    return najkrotsza_dlugosc